In [7]:
import pandas as pd

In [8]:
# import dataset
df = pd.read_csv(r'C:\Users\kettin\Documents\Lomba\GEMASTIKSciPaper\Dataset\ENSO_dataset_balanced.csv')
print(df.head())

     0         1    2    3    4    5         6         7         8         9  \
0  0.0  0.923388  0.0  0.0  0.0  0.0  2.990105  0.017291  1.682632  4.534331   
1  0.0  1.929875  0.0  0.0  0.0  0.0  5.562883  0.777918  0.464271  5.591380   
2  0.0  0.543871  0.0  0.0  0.0  0.0  3.117626  1.129129  0.000000  6.814785   
3  0.0  2.290855  0.0  0.0  0.0  0.0  1.345732  0.800221  0.000000  6.536081   
4  0.0  4.255888  0.0  0.0  0.0  0.0  0.000000  0.707398  0.000000  7.896858   

   ...       119  120  121  122  123  124       125       126  127  NINO3.4  
0  ...  0.155618  0.0  0.0  0.0  0.0  0.0  6.695034  1.521119  0.0    -1.55  
1  ...  1.411000  0.0  0.0  0.0  0.0  0.0  6.529056  2.613518  0.0    -1.78  
2  ...  1.029515  0.0  0.0  0.0  0.0  0.0  5.309260  2.167389  0.0    -1.38  
3  ...  1.345663  0.0  0.0  0.0  0.0  0.0  4.031511  2.411777  0.0    -1.90  
4  ...  2.738996  0.0  0.0  0.0  0.0  0.0  1.775552  4.273332  0.0    -1.74  

[5 rows x 129 columns]


In [9]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Pisahkan fitur dan target dari df
X = df.drop(columns=['NINO3.4']).values  # semua kolom kecuali target
y = df['NINO3.4'].values  # target tetap apa adanya

# Normalisasi ulang fitur
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Gabung kembali jadi DataFrame
df = pd.DataFrame(X_scaled, columns=df.columns[:-1])  # fitur dengan nama kolom aslinya
df['NINO3.4'] = y  # target tetap sama (nggak dinormalisasi)

print("Normalisasi selesai. Fitur sekarang 0–1, target tetap asli.")
print(f"Ukuran dataset: {df.shape[0]} sampel, {df.shape[1]-1} fitur + 1 target (NINO3.4)")


Normalisasi selesai. Fitur sekarang 0–1, target tetap asli.
Ukuran dataset: 1221 sampel, 128 fitur + 1 target (NINO3.4)


In [10]:
print(df.head())

     0         1    2    3    4    5         6         7         8         9  \
0  0.0  0.088295  0.0  0.0  0.0  0.0  0.227698  0.001255  0.197671  0.335540   
1  0.0  0.184535  0.0  0.0  0.0  0.0  0.423616  0.056479  0.054541  0.413761   
2  0.0  0.052005  0.0  0.0  0.0  0.0  0.237408  0.081978  0.000000  0.504293   
3  0.0  0.219052  0.0  0.0  0.0  0.0  0.102478  0.058099  0.000000  0.483669   
4  0.0  0.406950  0.0  0.0  0.0  0.0  0.000000  0.051359  0.000000  0.584366   

   ...       119  120  121  122  123  124       125       126  127  NINO3.4  
0  ...  0.010775  0.0  0.0  0.0  0.0  0.0  0.557715  0.144249  0.0    -1.55  
1  ...  0.097702  0.0  0.0  0.0  0.0  0.0  0.543889  0.247843  0.0    -1.78  
2  ...  0.071287  0.0  0.0  0.0  0.0  0.0  0.442276  0.205536  0.0    -1.38  
3  ...  0.093177  0.0  0.0  0.0  0.0  0.0  0.335836  0.228711  0.0    -1.90  
4  ...  0.189656  0.0  0.0  0.0  0.0  0.0  0.147909  0.405245  0.0    -1.74  

[5 rows x 129 columns]


In [11]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import TimeSeriesTransformerForPrediction, TimeSeriesTransformerConfig

# Gunakan hanya kolom target
target_series = df['NINO3.4'].values.astype(np.float32)


In [12]:
context_length = 120  # 10 tahun konteks
prediction_length = 24  # 2 tahun ke depan
stride = 12  # geser setahun sekali

In [13]:
class ENSOUnivariateDataset(Dataset):
    def __init__(self, series, context_length, prediction_length, stride=1):
        self.series = series
        self.context_length = context_length
        self.prediction_length = prediction_length
        self.stride = stride
        self.indices = self.create_indices()

    def create_indices(self):
        total = self.context_length + self.prediction_length
        return [i for i in range(0, len(self.series) - total, self.stride)]

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        i = self.indices[idx]
        past_values = self.series[i : i + self.context_length]
        future_values = self.series[i + self.context_length : i + self.context_length + self.prediction_length]
        return {
            "past_values": torch.tensor(past_values).unsqueeze(-1),  # (context_length, 1)
            "future_values": torch.tensor(future_values).unsqueeze(-1),
            "past_observed_mask": torch.ones(self.context_length, 1),  # all observed
        }


In [14]:
model = TimeSeriesTransformerForPrediction.from_pretrained("huggingface/time-series-transformer-tourism-monthly")
model.eval()

c:\Users\kettin\miniconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kettin\.cache\huggingface\hub\models--huggingface--time-series-transformer-tourism-monthly. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. 

TimeSeriesTransformerForPrediction(
  (model): TimeSeriesTransformerModel(
    (scaler): TimeSeriesMeanScaler()
    (embedder): TimeSeriesFeatureEmbedder(
      (embedders): ModuleList(
        (0): Embedding(366, 6)
      )
    )
    (encoder): TimeSeriesTransformerEncoder(
      (value_embedding): TimeSeriesValueEmbedding(
        (value_projection): Linear(in_features=27, out_features=26, bias=False)
      )
      (embed_positions): TimeSeriesSinusoidalPositionalEmbedding(48, 26)
      (layers): ModuleList(
        (0-1): 2 x TimeSeriesTransformerEncoderLayer(
          (self_attn): TimeSeriesTransformerAttention(
            (k_proj): Linear(in_features=26, out_features=26, bias=True)
            (v_proj): Linear(in_features=26, out_features=26, bias=True)
            (q_proj): Linear(in_features=26, out_features=26, bias=True)
            (out_proj): Linear(in_features=26, out_features=26, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((26,), eps=1e-05, element

In [15]:
train_size = int(len(target_series) * 0.8)
train_series = target_series[:train_size]
test_series = target_series[train_size:]

test_dataset = ENSOUnivariateDataset(test_series, context_length, prediction_length, stride)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


In [16]:
predictions = []
targets = []

with torch.no_grad():
    for batch in test_loader:
        for key in batch:
            batch[key] = batch[key]
        output = model(**batch)
        mean_prediction = output.distributions.mean.squeeze().numpy()
        target = batch["future_values"].squeeze().numpy()
        predictions.append(mean_prediction)
        targets.append(target)

y_pred = np.concatenate(predictions)
y_true = np.concatenate(targets)


TypeError: TimeSeriesTransformerForPrediction.forward() missing 1 required positional argument: 'past_time_features'